In [5]:
import torch 
from torch import nn, optim 
from torchvision import datasets, transforms 
from torch.utils.data import DataLoader

# Use of GPU otherwise CPU 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# List of transformation
tf = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Opens en applies transformations established previously
train_dl = DataLoader(
    datasets.ImageFolder('data/Training', tf),
    batch_size=32, shuffle=True, num_workers=4, pin_memory=True 
)

test_dl = DataLoader(
    datasets.ImageFolder('data/Testing', tf),
    batch_size=32, shuffle=True, num_workers=4, pin_memory=True 
)

In [3]:
model = nn.Sequential(
    nn.Conv2d(3, 32, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2),
    nn.Conv2d(32, 64, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2),
    nn.Conv2d(64, 128, 3, 1, 1), nn.ReLU(), nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(128 * 16 * 16, 256), nn.ReLU(), nn.Dropout(0.5),
    nn.Linear(256, 4) # There are four classes
).to(device)

In [4]:
opt = optim.AdamW(model.parameters(), 1e-4)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
model.train()

for epoch in range(25) :
    running_loss = 0

    for x, y in train_dl :
        opt.zero_grad()

        loss = loss_fn(model(x.to(device)), y.to(device))
        loss.backward()

        running_loss += loss 

        opt.step()

    print(f'Epoch {epoch + 1}: Loss was {running_loss}')

        

c:\Users\qevan\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 1: Loss was 152.30270385742188
Epoch 2: Loss was 103.6070556640625
Epoch 3: Loss was 87.07495880126953
Epoch 4: Loss was 74.62388610839844
Epoch 5: Loss was 65.86027526855469
